In [1]:
import pandas as pd
from datetime import date, timedelta
import numpy as np
import math
from scipy.stats import norm
from openpyxl import workbook
from IPython.display import display, HTML

import itertools

In [2]:
# some metadata for columns, file names, date
file_name = 'OptionsPivotTables.xlsm'
input_cols = ['Underlying', 'Underlying Price', 'Div. Yield', 'Security Type',
       'Currency', 'Position', 'Strike', 'CallPut', 'Maturity', 'Imp. Vol',
       'Risk-Free Rate', 'CounterParty']
greek_cols = ['$Delta (ESP)', '$Gamma', 'Vega 1%', 'Volga',
       'Vanna', 'Rho (1%)', 'Total $Delta', 'Total $Gamma', 'Total Vega 1%',
       'Total Volga', 'Total Vanna', 'Total Rho (1%)']

dt = pd.to_datetime(date(2010, 1, 10))

# read in the options data
data = pd.read_excel(file_name, sheet_name='OptionsData', skiprows=[0, 1, 2])

In [3]:
# reduce the data to just input information
input_data = data[input_cols]
input_data.index.name = 'ix'

### Question 1 

The functions below use the standaed Black-Scholes Model with Dividend Yield $q$

##### $d_1$ and $d_2$:
$$
d_1 = \frac{\ln\left(\frac{S_0}{K}\right) + \left(r - q + \frac{\sigma^2}{2}\right)T}{\sigma \sqrt{T}}
$$
$$
d_2 = d_1 - \sigma \sqrt{T}
$$

##### Call Option Price $C$:
$$
C = e^{-qT}S_0 N(d_1) - e^{-rT}K N(d_2)
$$

##### Put Option Price $P$:
$$
P = e^{-rT}K N(-d_2) - e^{-qT}S_0 N(-d_1)
$$

#### Variables:
- $S_0$: Current price of the underlying asset
- $K$: Strike price of the option
- $T$: Time to maturity (in years)
- $r$: Risk-free interest rate (annualized)
- $q$: Continuous dividend yield (annualized)
- $\sigma$: Volatility of the underlying asset (annualized)
- $N(\cdot)$: Cumulative distribution function of the standard normal distribution
- $N'(d_1) = \frac{1}{\sqrt{2\pi}} e^{-\frac{d_1^2}{2}}$: Standard normal probability density function

##### The futures price $F_0$ is given by:

$$
F_0 = S_0 e^{(r - q)T}
$$


In [4]:
def calc_d1(S, r, sigma, K, T, q):
    return (np.log(S/K) + (r - q + (sigma**2)/2) * T) / (sigma * np.sqrt(T))

def calc_d2(d1, sigma, T):
    
    return d1 - (sigma * np.sqrt(T))
    
def bs_option_price(S, r, sigma, K, T, q, is_call):
    call_put_scalar = 1 if is_call else -1
    d1 = calc_d1(S=S, r=r, sigma=sigma, K=K, T=T, q=q)
    d2 = calc_d2(d1=d1, sigma=sigma, T=T)
    px = call_put_scalar * S * np.exp(-q * T) * norm.cdf(call_put_scalar * d1, loc=0, scale=1) - call_put_scalar * np.exp((-r) * T) * K * norm.cdf(call_put_scalar * d2, loc=0, scale=1)
    
    return px

def futures_price(S, r, q, T):
    px = np.exp((r - q) * T) * S
    return px

def instrument_price(row, price_scenarios=np.array([0]), vol_scenarios=np.array([0])):
    sec_type = row['Security Type']
    call_put = row['CallPut']
    S = row['Underlying Price'] * (1 + price_scenarios)
    r = row.fillna(0)['Risk-Free Rate']
    sigma = row['Imp. Vol'] + vol_scenarios
    K = row['Strike']
    T = (row['Maturity'] - dt).days / 365
    q = row['Div. Yield']    
    
    match sec_type.lower():
        case 'future':
            px = futures_price(S=S, r=r, q=q, T=T)
        case 'option':
            px = bs_option_price(S=S, r=r, sigma=sigma, K=K, T=T, q=q, is_call=call_put.lower() == 'call')
        case _:
            raise ValueError(f'Unrecognized security type {sec_type}')
    return px

In [5]:
# create the 49 different combinations based on underlying price movement and implied vol movement
price_scenarios = [-0.2, -0.1, -0.05, 0, 0.05, 0.1, 0.2]
vol_scenarios = [-0.05, -0.02, 0, 0.02, 0.05, 0.1, 0.2]
scenarios = np.array(list(itertools.product(price_scenarios, vol_scenarios))).T

# format column names
scenarios_df = pd.DataFrame(scenarios)
scenario_cols = 'Und ' + scenarios_df.loc[0, :].mul(100).astype(int).astype(str) + '%' + ', ImpVol ' + scenarios_df.loc[1, :].mul(100).astype(int).astype(str) + '%'

# apply the calculation to each row
# note - this is vectorized across rows
scenario_result = pd.DataFrame(
    input_data.apply(instrument_price, args=(scenarios[0, :], scenarios[1, :]), axis=1).tolist(),
    index=input_data.index, columns=scenario_cols)

# save off stacked results
# meaning 
scenarios_stacked = scenario_result.copy()
scenarios_stacked.columns = pd.MultiIndex.from_arrays(scenarios, names=['und_chg', 'vol_chg'])
scenarios_stacked = scenarios_stacked.stack(['und_chg', 'vol_chg']).sort_index().rename('Price')
scenarios_stacked.index.name = 'ix'
scenarios_stacked = input_data.join(scenarios_stacked, how='right').reset_index().drop(columns=['ix'])

## save output to table for writing to excel
input_data.loc[:, scenario_cols] = scenario_result
input_data.loc[:, scenario_cols] = input_data.loc[:, scenario_cols].sub(
    input_data.loc[:, 'Und 0%, ImpVol 0%'], axis=0).mul(
    input_data.loc[:, 'Position'], axis=0
)

C:\Users\yrevtsov\AppData\Local\Temp\ipykernel_18328\1988830168.py:20: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  scenarios_stacked = scenarios_stacked.stack(['und_chg', 'vol_chg']).sort_index().rename('Price')


### Question 2.1
Since we were asked to provide screenshots of an Excel file I decided it's cleaner to just have tables in python

In [6]:
scenario_to_show = 'Und -10%, ImpVol 10%'
for und in input_data.Underlying.unique():
    print('\n')
    display(HTML(f"<b>{und}</b>"))
    display(input_data[
        (input_data.Underlying == und)
    ].groupby(['Strike', 'Maturity'])[[scenario_to_show]].sum().unstack('Maturity').round(2))

Und -10%, ImpVol 10%                      
Maturity           2010-03-19 2010-06-18 2010-09-17
Strike                                             
902.0                     NaN    2689.16        NaN
935.0                10489.31        NaN        NaN
1001.0               -1927.96   -2042.06   -1477.98
1034.0               -8206.47   -1468.63        NaN
1067.0                    NaN        NaN   -2945.81
1100.0                2739.77   -2963.12   -4424.88
1232.0                -567.05     259.74     263.02
1298.0                    NaN    -206.99        NaN
1331.0                    NaN    -129.74        NaN

Und -10%, ImpVol 10%                      
Maturity           2010-03-19 2010-06-18 2010-09-17
Strike                                             
2469.0                9175.39    1525.57        NaN
2556.0               -4816.47        NaN     905.63
2731.0                    NaN       0.00   -4125.73
2818.0                2959.07   -8276.56        NaN
2905.0                    NaN   -7730.29        NaN
2992.0               -7712.93        NaN        NaN
3079.0               -1801.67    2105.23        NaN
3166.0                 940.64     320.23        NaN
3254.0               -1385.86     470.85        NaN
3341.0                    NaN     335.28   -5552.20
3428.0                    NaN    -227.98        NaN

Und -10%, ImpVol 10%                      
Maturity           2010-03-19 2010-06-18 2010-09-17
Strike                                             
7617.0                    NaN        NaN   -2443.44
8519.0               25415.41    2654.48        NaN
8819.0                    NaN        NaN  -12654.90
9120.0              -26731.72  -15526.18        NaN
9721.0                    NaN        NaN  -15310.30
10022.0              -1843.11  -34574.30        NaN
10323.0             -11126.63    2793.59        NaN
11225.0                   NaN   -7555.64        NaN
11525.0                   NaN    2319.81        NaN

### Question 2.2
The functions use standaed Black-Scholes Greeks formulas:

#### Delta ($\Delta$) in dollars:
For a **call** option:
$$
\Delta_{\text{call}} = e^{-qT} S_0 N(d_1) 
$$

For a **put** option:
$$
\Delta_{\text{put}} = e^{-qT} S_0 \left(N(d_1) - 1\right)
$$

#### Gamma ($\Gamma$) in dollars:
$$
\Gamma = e^{-qT} S_0 \frac{N'(d_1)}{2 \sigma \sqrt{T}}
$$

#### Vega ($\text{Vega}$):
$$
\text{Vega} = S_0 e^{-qT} \sqrt{T} N'(d_1)
$$

#### Volga (Second derivative with respect to volatility):
$$
\text{Volga} = \text{Vega} \cdot \frac{d_1 d_2}{\sigma}
$$

#### Vanna (Cross-derivative with respect to volatility and spot price):
$$
\text{Vanna} = T e^{-qT} S_0 N'(d_1) \left( 1 - \frac{d_1}{\sigma} \right)
$$

In [7]:
def bs_delta(S, r, sigma, K, T, q, is_call):
    d1 = calc_d1(S=S, r=r, sigma=sigma, K=K, T=T, q=q)
    if is_call:
        delta = np.exp(-q * T) * norm.cdf(d1)
    else:
        delta = np.exp(-q * T) * (norm.cdf(d1) - 1)
    return delta * S

def bs_gamma(S, r, sigma, K, T, q):
    d1 = calc_d1(S=S, r=r, sigma=sigma, K=K, T=T, q=q)
    # gamma = np.exp(-q * T) * norm.cdf(d1) / (sigma * S * np.sqrt(T))
    gamma = np.exp(-q * T) * norm.pdf(d1) * S / (2 * sigma * np.sqrt(T))
    return gamma
    
def bs_vega(S, r, sigma, K, T, q, pct_change=1):
    d1 = calc_d1(S=S, r=r, sigma=sigma, K=K, T=T, q=q)
    vega = np.exp(-q * T) * S * np.sqrt(T) * norm.pdf(d1)
    return vega * pct_change
    
def bs_volga(S, r, sigma, K, T, q):
    vega = bs_vega(S=S, r=r, sigma=sigma, K=K, T=T, q=q)
    d1 = calc_d1(S=S, r=r, sigma=sigma, K=K, T=T, q=q)
    d2 = calc_d2(d1=d1, sigma=sigma, T=T)
    volga = vega * d1 * d2 / sigma
    return volga

def bs_vanna(S, r, sigma, K, T, q):
    vega = bs_vega(S=S, r=r, sigma=sigma, K=K, T=T, q=q)
    d1 = calc_d1(S=S, r=r, sigma=sigma, K=K, T=T, q=q)
    # vanna = vega * (1 - d1/sigma)
    vanna = (vega / S) * (1 - d1 / (sigma * np.sqrt(T)))
    return vanna * S / (2 * 100)
    
def greeks(row):
    sec_type = row['Security Type']
    call_put = row['CallPut']
    S = row['Underlying Price']
    r = row.fillna(0)['Risk-Free Rate']
    sigma = row['Imp. Vol']
    K = row['Strike']
    T = (row['Maturity'] - dt).days / 365
    q = row['Div. Yield']    

    match sec_type.lower():
        case 'future':
            greeks = np.array([
                np.exp((r - q) * T),
                0,
                0,
                0,
                0,
            ])
        case 'option':
            is_call = call_put.lower() == 'call'
            greeks = np.array([
                bs_delta(S=S, r=r, sigma=sigma, K=K, T=T, q=q, is_call=is_call),
                bs_gamma(S=S, r=r, sigma=sigma, K=K, T=T, q=q),
                bs_vega(S=S, r=r, sigma=sigma, K=K, T=T, q=q, pct_change=0.01),
                bs_volga(S=S, r=r, sigma=sigma, K=K, T=T, q=q),
                bs_vanna(S=S, r=r, sigma=sigma, K=K, T=T, q=q),
            ])

    return greeks

# calculate the greeks for each position (row)
greek_cols = ['Delta$', 'Gamma$', 'Vega1%', 'Volga', 'Vanna']
input_data.loc[:, greek_cols] = pd.DataFrame(
    input_data.apply(greeks, axis=1).tolist(),
    index=input_data.index, columns=greek_cols)

In [8]:
for und in input_data.Underlying.unique():
    for greek in greek_cols:
        print('\n')
        display(HTML(f"<b>{und}</b>"))
        display(input_data[
            (input_data.Underlying == und)
        ].groupby(['Strike', 'Maturity'])[[greek]].sum().unstack('Maturity').round(2))

Delta$                      
Maturity 2010-03-19 2010-06-18 2010-09-17
Strike                                   
902.0           NaN    -497.72        NaN
935.0       -455.08        NaN        NaN
1001.0       463.34    -347.74    -354.88
1034.0     -1122.04    1004.10        NaN
1067.0          NaN        NaN     208.74
1100.0       651.23      94.88    -486.43
1232.0       586.25     366.66     397.94
1298.0          NaN     275.72        NaN
1331.0          NaN     238.28        NaN

Gamma$                      
Maturity 2010-03-19 2010-06-18 2010-09-17
Strike                                   
902.0           NaN     963.56        NaN
935.0       1364.93        NaN        NaN
1001.0      1841.89     701.72     605.34
1034.0      3133.48    2342.90        NaN
1067.0          NaN        NaN    1469.87
1100.0      3701.54    1825.25     787.76
1232.0      2202.52     915.19     824.39
1298.0          NaN     804.40        NaN
1331.0          NaN     739.02        NaN

Vega1%                      
Maturity 2010-03-19 2010-06-18 2010-09-17
Strike                                   
902.0           NaN       4.35        NaN
935.0          2.71        NaN        NaN
1001.0         3.23       2.57       3.24
1034.0         5.21       8.08        NaN
1067.0          NaN        NaN       6.96
1100.0         5.64       5.71       3.55
1232.0         3.11       2.63       3.38
1298.0          NaN       2.30        NaN
1331.0          NaN       2.12        NaN

Volga                      
Maturity 2010-03-19 2010-06-18 2010-09-17
Strike                                   
902.0           NaN     251.69        NaN
935.0        242.32        NaN        NaN
1001.0       135.53      55.64      46.30
1034.0       103.36      71.89        NaN
1067.0          NaN        NaN     -20.78
1100.0       -10.63     -22.18     -19.98
1232.0       399.33     215.75     226.16
1298.0          NaN     411.50        NaN
1331.0          NaN     497.82        NaN

Vanna                      
Maturity 2010-03-19 2010-06-18 2010-09-17
Strike                                   
902.0           NaN      -2.56        NaN
935.0         -3.45        NaN        NaN
1001.0        -2.82      -0.90      -0.62
1034.0        -2.93      -1.52        NaN
1067.0          NaN        NaN       0.52
1100.0         1.56       1.54       0.93
1232.0         7.46       3.87       4.01
1298.0          NaN       4.68        NaN
1331.0          NaN       4.85        NaN

...

Delta$                      
Maturity 2010-03-19 2010-06-18 2010-09-17
Strike                                   
7617.0          NaN        NaN   -2064.28
8519.0     -4127.13   -2519.87        NaN
8819.0          NaN        NaN   -5848.48
9120.0     -8595.33   -6328.06        NaN
9721.0          NaN        NaN   -7971.38
10022.0     6065.29   -9054.50        NaN
10323.0     -691.49    4942.98        NaN
11225.0         NaN   -3200.49        NaN
11525.0         NaN    5912.91        NaN

Gamma$                      
Maturity 2010-03-19 2010-06-18 2010-09-17
Strike                                   
7617.0          NaN        NaN    2994.65
8519.0     12378.61    4993.36        NaN
8819.0          NaN        NaN    9813.03
9120.0     25067.36   12773.46        NaN
9721.0          NaN        NaN   13408.07
10022.0    33725.56   16672.44        NaN
10323.0    23192.23    8695.40        NaN
11225.0         NaN   16848.33        NaN
11525.0         NaN   15948.93        NaN

Vega1%                      
Maturity 2010-03-19 2010-06-18 2010-09-17
Strike                                   
7617.0          NaN        NaN      23.60
8519.0        24.64      21.06        NaN
8819.0          NaN        NaN      56.84
9120.0        44.10      46.98        NaN
9721.0          NaN        NaN      63.83
10022.0       51.52      52.29        NaN
10323.0       34.35      26.33        NaN
11225.0         NaN      48.37        NaN
11525.0         NaN      45.59        NaN

Volga                      
Maturity 2010-03-19 2010-06-18 2010-09-17
Strike                                   
7617.0          NaN        NaN    1147.23
8519.0      2229.64    1013.82        NaN
8819.0          NaN        NaN    1431.65
9120.0      1903.28    1059.00        NaN
9721.0          NaN        NaN    -149.52
10022.0      -98.39    -205.01        NaN
10323.0      193.41      26.43        NaN
11225.0         NaN    3814.84        NaN
11525.0         NaN    5622.54        NaN

Vanna                      
Maturity 2010-03-19 2010-06-18 2010-09-17
Strike                                   
7617.0          NaN        NaN      -8.47
8519.0       -31.50     -11.49        NaN
8819.0          NaN        NaN     -15.86
9120.0       -39.05     -16.79        NaN
9721.0          NaN        NaN       3.53
10022.0       12.88      13.07        NaN
10323.0       25.85      13.99        NaN
11225.0         NaN      70.06        NaN
11525.0         NaN      79.32        NaN

In [9]:
input_data.to_excel('options_scenarios.xlsx')

### Question 2.3
Creating a pivot table of PnL values as functio nof shift in the underlying and shift in implied vol is not possible given the current shape of data. One way to reshape the data is to have a row for each scenario. So the table would contain the security information, then new columns such as "Und Shift", "ImpVol Shift", and then the calculated PnL. This would of course mean that instead of having one row for each unique derivative you would have 49 rows.

### Question 3.1
Given a scenario of a sudden shock/crisis in the market, let's discuss its impact on implied volatility. 

During market turbulence, short-term and long-term implied volatilities often respond differently due to shifts in risk perception, liquidity, and supply-demand imbalances in derivatives markets.

Short-term implied volatilities typically spike more than long-term volatilities due to immediate uncertainty and liquidity stress. This leads to a steepening of the volatility term structure. After a crisis, short-term volatilities tend to normalize faster, leading to a flattening of the term structure.

Out-of-the-money puts often experience larger increases in implied volatility than calls during a crisis due to demand for downside protection.

### Question 3.2

A portfolio that is delta, gamma, vega, and vanna neutral has effectively neutralized its first and *some* second-order exposures:
- Delta neutrality: no directional bias to small changes in the underlying 
- Gamma neutrality: no curvature exposure to larger moves in 
- Vega neutrality: no first-order exposure to parallel shifts in implied volatility
- Vanna neutrality: no sensitivity to the interaction of changes in the underlying and implied vol

However, the problem states that the portfolio is short skew. In Greek terms, this typically indicates exposure to higher-order Greeks that capture the shape or slope of the implied volatility across strikes referred to as volga.

Short skew means the portfolio loses if the difference between out-of-the-money (OTM) implied volatilities and at-the-money (ATM) implied volatilities becomes larger (steepening skew), and it profits if that difference narrows (flattening skew). Standard scenario analyses that simply: move the underlying up or down, or shift the entire implied volatility surface in parallel will not necessarily reveal risk to skew-specific changes. Such scenarios do not precisely model how OTM options (puts or calls) might re-price relative to ATM options, nor do they capture localized distortions in the surface.

To see the true risk when you are short skew, you need scenario analyses that focus on non-parallel changes in implied volatility across strikes (and potentially maturities).

- Skew Twist/Flattening/Steepening: Increase implied volatility in far OTM puts and decrease it in ATM/near-ATM calls (or vice versa). These scenarios measure how the portfolio P/L responds to the slope of the smile changing.
- Local Distortions: Change implied volatility in just one region of the strike surface (e.g., OTM puts) while holding others constant. Assess how an uneven re-pricing of option strikes affects P/L.
- Jump or Crash Scenarios: When markets crash or spike, OTM options often experience more dramatic IV changes (a “skew blowout”), which is exactly where a short skew position is most vulnerable.
- Term Structure/Skew Interactions: Alter the shape of implied vol across maturities (term structure) and across strikes (skew) simultaneously, since the risk can differ across tenors.
- These stress tests go beyond standard delta/gamma/vega shifts to isolate the portfolio’s volga (or higher-order) sensitivity to skew.

In [10]:
underlying = 'SPX Index'
option_type = 'Call'
scenarios_stacked[
    (scenarios_stacked.Underlying == underlying)
    & (scenarios_stacked.CallPut == option_type)
]

,und_chg,vol_chg,Underlying,Underlying Price,Div. Yield,Security Type,Currency,Position,Strike,CallPut,Maturity,Imp. Vol,Risk-Free Rate,CounterParty,Price
245,-0.2,-0.05,SPX Index,1100,0.02,Option,USD,25,1232.0,Call,2010-03-19,0.379364,0.0110,Exch,0.427284
246,-0.2,-0.02,SPX Index,1100,0.02,Option,USD,25,1232.0,Call,2010-03-19,0.379364,0.0110,Exch,0.830827
247,-0.2,0.00,SPX Index,1100,0.02,Option,USD,25,1232.0,Call,2010-03-19,0.379364,0.0110,Exch,1.207032
248,-0.2,0.02,SPX Index,1100,0.02,Option,USD,25,1232.0,Call,2010-03-19,0.379364,0.0110,Exch,1.677899
249,-0.2,0.05,SPX Index,1100,0.02,Option,USD,25,1232.0,Call,2010-03-19,0.379364,0.0110,Exch,2.572551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1416,0.2,0.00,SPX Index,1100,0.02,Option,USD,-50,1067.0,Call,2010-09-17,0.345443,0.0175,Darklays Capital,292.323572
1417,0.2,0.02,SPX Index,1100,0.02,Option,USD,-50,1067.0,Call,2010-09-17,0.345443,0.0175,Darklays Capital,298.239039
1418,0.2,0.05,SPX Index,1100,0.02,Option,USD,-50,1067.0,Call,2010-09-17,0.345443,0.0175,Darklays Capital,307.391701
1419,0.2,0.10,SPX Index,1100,0.02,Option,USD,-50,1067.0,Call,2010-09-17,0.345443,0.0175,Darklays Capital,323.231953


### Question 4
Again, functions below use the standard formulas which are written out here

#### Value at Risk (VaR) and Expected Shortfall (CVaR)

##### For the **Normal Distribution**:

- **Value at Risk (VaR):**
$$
\text{VaR}_{\text{Normal}}(\alpha) = \mu + z_\alpha \sigma
$$

- **Expected Shortfall (CVaR):**
$$
\text{ES}_{\text{Normal}}(\alpha) = \mu + \sigma \frac{\phi(z_\alpha)}{1 - \alpha}
$$

##### For the **Student's $t$-Distribution**:

- **Value at Risk (VaR):**
$$
\text{VaR}_{t}(\alpha) = \mu + t_{\nu, \alpha} \frac{\sigma}{\sqrt{\nu / (\nu - 2)}}
$$

- **Expected Shortfall (CVaR):**
$$
\text{ES}_{t}(\alpha) = \mu + \frac{\sigma}{\sqrt{\nu / (\nu - 2)}} \cdot \frac{t_{\nu, \alpha} \cdot (\nu + t_{\nu, \alpha}^2)}{(\nu - 1)(1 - \alpha)}
$$


#### Variables:
- $\alpha$: Confidence level (e.g., 0.95 or 0.99)
- $\mu$: Mean of the portfolio returns
- $\sigma$: Standard deviation of the portfolio returns
- $z_\alpha$: Quantile of the standard normal distribution at the confidence level $\alpha$ (e.g., $z_{0.95}$ or $z_{0.99}$)
- $t_{\nu, \alpha}$: Quantile of the Student's $t$-distribution at the confidence level $\alpha$ with $\nu$ degrees of freedom
- $\phi(z)$: Probability density function (PDF) of the standard normal distribution
- $\nu$: Degrees of freedom for the Student's $t$-distribution


In [11]:
from scipy.stats import norm, t

def calc_var_norm(mu, sigma, alpha):
    var = mu + sigma * norm.ppf(q=alpha)
    return var

def calc_var_t(mu, sigma, alpha, df):
    var = mu + sigma * t.ppf(q=alpha, df=df)
    return var

def calc_es_norm(mu, sigma, alpha):
    es = mu + sigma * norm.pdf(norm.ppf(q=alpha)) / (1 - alpha)
    return es

def calc_es_t(mu, sigma, alpha, df):
    quantile = t.ppf(q=alpha, df=df)
    es_tilda = mu + t.pdf(quantile, df=df) / (1 - alpha) * (df + quantile**2) / (df - 1)
    es = mu + sigma * es_tilda
    return es

In [12]:
# mean and standard deviation inputs
mu = 0
sigma = 0.31 / np.sqrt(250)
# alpha inputs
alphas = [0.90, 0.95, 0.975, 0.99, 0.995, 0.999, 0.9999, 0.99999, 0.999999]
risk_measures = pd.DataFrame(data=alphas, columns=['alpha'])

risk_measures.loc[:, 'VaR.norm'] = risk_measures.apply(lambda row: calc_var_norm(mu, sigma, row.alpha), axis=1)
risk_measures.loc[:, 'VaR.t4'] = risk_measures.apply(lambda row: calc_var_t(mu, sigma, row.alpha, df=4), axis=1)
risk_measures.loc[:, 'ES.norm'] = risk_measures.apply(lambda row: calc_es_norm(mu, sigma, row.alpha), axis=1)
risk_measures.loc[:, 'ES.t4'] = risk_measures.apply(lambda row: calc_es_t(mu, sigma, row.alpha, df=4), axis=1)
risk_measures = risk_measures.set_index('alpha')
risk_measures *= 1e4
risk_measures.loc[:, 'Ratio.norm'] = risk_measures['ES.norm'] / risk_measures['VaR.norm']
risk_measures.loc[:, 'Ratio.t4'] = risk_measures['ES.t4'] / risk_measures['VaR.t4']

#### Results
VaR and ES from normal and t-distribution with4 degress of freedom

In [13]:
risk_measures.round(2)

,VaR.norm,VaR.t4,ES.norm,ES.t4,Ratio.norm,Ratio.t4
alpha,,,,,,
0.900000,251.26,300.60,344.08,490.02,1.37,1.63
0.950000,322.49,417.97,404.42,627.96,1.25,1.50
0.975000,384.27,544.35,458.35,782.98,1.19,1.44
0.990000,456.11,734.63,522.55,1023.55,1.15,1.39
0.995000,505.02,902.68,567.00,1240.05,1.12,1.37
0.999000,605.87,1406.38,660.16,1899.09,1.09,1.35
0.999900,729.15,2555.40,776.10,3420.50,1.06,1.34
0.999990,836.18,4574.54,878.11,6106.84,1.05,1.33
0.999999,931.96,8151.80,970.18,10873.26,1.04,1.33


The ES to VaR ratio is larger for t-distribution than for normal distribution. This aligns with intuition; since t-distribution has fatter tails, the expected value of those tails (which is essentially what epected shortfall is) is much larger than expected value of the same quantile tail 